In [31]:
import io
import os
import math
import base64
import datetime
import rdkit
import ipywidgets as widgets
import numpy as np
import pandas as pd
import IPython

In [32]:
def rdimage(mol: rdkit.Chem.Mol) -> bytes:
    """Returns an image of a molecule as bytes."""
    pil_image = rdkit.Chem.Draw.MolToImage(mol)
    buf = io.BytesIO()
    pil_image.save(buf, format='png')
    return buf.getvalue()

In [33]:
def df_from_uploader(uploader: widgets.FileUpload):
    uploaded_file = next(iter(uploader.value.values()))  # ipywidgets 7.6
    # uploaded_file = uploader.value[0]  # ipywidgets 8.0

    df = pd.read_csv(io.BytesIO(uploaded_file["content"]))
    filename = uploaded_file.get("metadata", {}).get("name", "no_name")
    return df, filename

In [34]:
def add_rdmol(df):
    df["mol"] = df["SMILES"].apply(rdkit.Chem.MolFromSmiles)
    return df

In [35]:
def add_molwidget(df):
    df["mol_widget"] = df["mol"].apply(
        lambda mol: widgets.Image(
            value=rdimage(mol),
            format="PNG",
            width=150,
            height=150,
        )
    )
    return df

In [36]:
def add_buttonwidget(df):
    df["button_widget"] = df["SMILES"].apply(
        lambda smi: widgets.ToggleButton(
            value=False, 
            description=smi,
            layout=widgets.Layout(height="auto", width="auto")
        )
    )
    return df

In [37]:
def add_duowidget(df):
    df["duo_widget"] = df.apply(
        axis=1, 
        func=lambda row: widgets.VBox(
            [row.mol_widget, row.button_widget],
            layout=widgets.Layout(margin="5px 5px 40px 5px")  # top, right, bottom, left
        )
    )
    return df

In [38]:
def add_gui_to_df(df):
    """Returns dataframe with all needed GUI widgets added in-place as columns."""
    return df.pipe(add_rdmol).pipe(add_molwidget).pipe(add_buttonwidget).pipe(add_duowidget)

In [39]:
def remove_gui_from_df(df):
    """Returns copy of the dataframe without GUI columns removed."""
    df["is_selected"] = df["button_widget"].map(lambda row: row.value)

    auxiliary_ui_columns = ["mol", "mol_widget", "button_widget", "duo_widget"]
    good_columns = df.columns.difference(auxiliary_ui_columns)
    return df[good_columns]

In [40]:
def make_grid(df):
    """Returns Grid widget populated by molecules."""
    df = add_gui_to_df(df)
    N_COLS = 4
    N_ROWS = math.ceil(len(df) / N_COLS)
    grid = widgets.GridspecLayout(N_ROWS, N_COLS)
    for i in range(len(df)):
        (row, col) = np.unravel_index(i, (N_ROWS, N_COLS))
        grid[row, col] = df["duo_widget"][i]
    return grid

In [41]:
# https://stackoverflow.com/a/42907645
def create_download_link(df, filename = "output.csv"):
    """Returns HTML widget with a download link, with dataframe embedded in the link as a CSV file."""
    csv = remove_gui_from_df(df).to_csv(index=False)
    payload = base64.b64encode(csv.encode()).decode()
    now = datetime.datetime.now().ctime()
    html = f'Output file: <a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{filename}</a>. Generated {now}.'
    return widgets.HTML(html)

In [42]:
def mkfname(ori_filename):
    """Makes a file name for the output from the original upload file name."""
    if ori_filename is None:
        return "output.csv"
    
    name = os.path.splitext(ori_filename)[0]
    return f"{name}_with_selection.csv"


def make_download_button(df, ori_filename=None):
    """Returns download button.
    
    Download button is wrapped in a Box.
    Initially, there is only one child in the box - the button.
    On click, the children of the Box are replaced with two children:
        - the original button,
        - a link to download a file.
    The link contains embedded a CSV file representation of the dataframe.
    """
    
    btn = widgets.Button(description = 'Generate output file') 
    vbox = widgets.VBox([btn])
    def on_click(b):
        vbox.children = tuple([btn, create_download_link(df, mkfname(ori_filename))])
        
    btn.on_click(on_click)
    return vbox

In [43]:
def make_app():
    """Returns App as a FileUploader widget.
    
    Initially, the widget contains only File Upload button.
    On click, the button is replaced by the following:
        - the upload button itself,
        - a grid of molecules ("the wall"),
        - a button to download CSV file with current selection.
    """
    
    description = widgets.HTML(
        'Upload a CSV file with SMILES column '
        '(see <a target="_blank" href="https://github.com/'
        'Augmented-Drug-Design-Human-in-the-Loop/'
        'mol-wall/tree/main/tests/data">example files</a>).'
    )
    uploader = widgets.FileUpload(
        accept='.csv,csv.gz',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False  # True to accept multiple files upload else False
    )
    app = widgets.VBox([description, uploader])
    
    def on_upload_change(change):
        uploader = change["owner"]
        df, filename = df_from_uploader(uploader)
        app.children = tuple([uploader, make_grid(df), make_download_button(df, filename)]) 

    uploader.observe(on_upload_change, names='_counter')  # https://stackoverflow.com/a/64357261
    return app

In [44]:
app = make_app()

In [45]:
app